In [1]:
%run ./config/config.ipynb

In [2]:
# using SQLContext to read parquet file
sqlContext = SQLContext(sc)

In [3]:
%run ./variables/path_variable.ipynb

In [4]:
%run ./variables/TempTable.ipynb

In [5]:
# to read parquet file
df_refined = spark.read.parquet(path_refined)

In [6]:
df_refined.count()

150000

In [7]:
df_refined.registerTempTable(TempTableRefined)

In [8]:
df_refined_sql = spark.sql("""
                          SELECT 
                                profile.name[0] AS name, 
                                profile.username[0] AS username, 
                                profile.sex[0] AS sex, 
                                profile.birthdate[0] AS birthdate, 
                                city, 
                                job, 
                                quantity AS visit_number, 
                                value AS total_clicks, 
                                last_access,
                                last_access AS access,
                                Host_name AS host, 
                                Domain_name AS domain 
                          FROM refined_delta 
                      """)

In [9]:
df_refined_sql.count()

150000

In [10]:
%run ./variables/today.ipynb

In [11]:
path_exist = DeltaTable.isDeltaTable(spark, path_trusted)

In [12]:
if path_exist == False:
    df_refined_select = df_refined_sql.select(df_refined_sql['*'])
else:
    df_refined_select = df_refined_sql.select(df_refined_sql['*']).filter(df_refined_sql['last_access'] == today)

In [13]:
df_refined_select.count()

10000

In [14]:
# Grava os arquivos na camada Processed
df_refined_select.write.format("delta") \
                       .mode("overwrite") \
                       .partitionBy('last_access') \
                       .save(path_trusted)

In [15]:
sc.stop()